# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [0]:
batch_size = 64
num_classes = 5
epochs = 25
lrate = 0.01
decay = lrate/epochs

### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [0]:
# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train[:, 0] < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test[:, 0] < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train[:, 0] >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test[:, 0] >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [69]:
print(x_train_lt5.shape,y_train_lt5.shape)
print(x_test_lt5.shape,y_test_lt5.shape)
print(x_train_gte5.shape,y_train_gte5.shape)
print(x_test_gte5.shape,y_test_gte5.shape)

(25000, 32, 32, 3) (25000,)
(5000, 32, 32, 3) (5000,)
(25000, 32, 32, 3) (25000,)
(5000, 32, 32, 3) (5000,)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
# normalize inputs from 0-255 to 0.0-1.0
x_train_lt5 = x_train_lt5.astype('float32')
x_test_lt5 = x_test_lt5.astype('float32')
x_train_gte5 = x_train_gte5.astype('float32')
x_test_gte5 = x_test_gte5.astype('float32')
x_train_lt5 /= 255
x_test_lt5 /= 255
x_train_gte5 /= 255
x_test_gte5 /= 255

# one hot encode outputs
y_train_lt5 = keras.utils.to_categorical(y_train_lt5, num_classes)
y_test_lt5 = keras.utils.to_categorical(y_test_lt5, num_classes)
y_train_gte5 = keras.utils.to_categorical(y_train_gte5, num_classes)
y_test_gte5 = keras.utils.to_categorical(y_test_gte5, num_classes)


### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [73]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=x_train_lt5.shape[1:], activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

# initiate SGD optimizer
opt = keras.optimizers.sgd(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

# Let's train the model using SGD
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

# Fit the model
model.fit(x_train_lt5, y_train_lt5, validation_data=(x_test_lt5, y_test_lt5), epochs=epochs, batch_size=64)

# Final evaluation of the model
scores = model.evaluate(x_test_lt5, y_test_lt5, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_7 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_8 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 64)       

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [74]:
# Freeze the layers except the last 5 layers
for layer in model.layers[:-5]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in model.layers:
    print(layer, layer.trainable)

<keras.layers.convolutional.Conv2D object at 0x7f2de1eceda0> False
<keras.layers.core.Dropout object at 0x7f2de2295c50> False
<keras.layers.convolutional.Conv2D object at 0x7f2de22954a8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f2de2517f60> False
<keras.layers.convolutional.Conv2D object at 0x7f2de3138438> False
<keras.layers.core.Dropout object at 0x7f2de31438d0> False
<keras.layers.convolutional.Conv2D object at 0x7f2de313ba90> False
<keras.layers.pooling.MaxPooling2D object at 0x7f2de3141860> False
<keras.layers.convolutional.Conv2D object at 0x7f2de315bf60> False
<keras.layers.core.Dropout object at 0x7f2de2f237b8> False
<keras.layers.convolutional.Conv2D object at 0x7f2de2f23208> False
<keras.layers.pooling.MaxPooling2D object at 0x7f2de2f55d68> False
<keras.layers.core.Flatten object at 0x7f2de2f53e48> False
<keras.layers.core.Dropout object at 0x7f2de2f43320> False
<keras.layers.core.Dense object at 0x7f2de24301d0> True
<keras.layers.core.Dropout object at 0x7f2de2f

In [75]:

# Create the model
freezemodel = Sequential()
 
# Add the original model convolutional base model
freezemodel.add(model)
  
# Show a summary of the model. Check the number of trainable parameters
freezemodel.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 5)                 2912549   
Total params: 2,912,549
Trainable params: 2,625,541
Non-trainable params: 287,008
_________________________________________________________________


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [76]:
# initiate SGD optimizer
opt = keras.optimizers.sgd(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

# Let's train the model using SGD
freezemodel.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
freezemodel.summary()

# Fit the model
freezemodel.fit(x_train_gte5, y_train_gte5, validation_data=(x_test_gte5, y_test_gte5), epochs=epochs, batch_size=batch_size)

# Final evaluation of the model
scores = freezemodel.evaluate(x_test_gte5, y_test_gte5, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 5)                 2912549   
Total params: 2,912,549
Trainable params: 2,625,541
Non-trainable params: 287,008
_________________________________________________________________
Train on 25000 samples, validate on 5000 samples
Epoch 1/25
25000/25000 [==============================] - 6s 260us/step - loss: 0.7566 - acc: 0.7370 - val_loss: 0.5138 - val_acc: 0.8100
Epoch 2/25
25000/25000 [==============================] - 6s 232us/step - loss: 0.5124 - acc: 0.8088 - val_loss: 0.4712 - val_acc: 0.8236
Epoch 3/25
25000/25000 [==============================] - 6s 231us/step - loss: 0.4634 - acc: 0.8267 - val_loss: 0.4462 - val_acc: 0.8384
Epoch 4/25
25000/25000 [==============================] - 6s 230us/step - loss: 0.4239 - acc: 0.8438 - val_loss: 0.4355 - val_acc: 0.8404
Epoch 5/25
25000/25000 [=======

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [81]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [82]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [83]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(twenty_train.data)
X_test_tfidf = vectorizer.transform(twenty_test.data)
dfTraining = pd.DataFrame(X_train_tfidf.todense(), columns=vectorizer.get_feature_names())
dfTesting = pd.DataFrame(X_test_tfidf.todense(), columns=vectorizer.get_feature_names())
print("Training features \n\n ", dfTraining.head())
print("\n Testing features \n\n ", dfTesting.head())

Training features 

           00  000  0000  0000001200  ...   zz  zzz   ªl  íålittin
0  0.000000  0.0   0.0         0.0  ...  0.0  0.0  0.0       0.0
1  0.000000  0.0   0.0         0.0  ...  0.0  0.0  0.0       0.0
2  0.000000  0.0   0.0         0.0  ...  0.0  0.0  0.0       0.0
3  0.000000  0.0   0.0         0.0  ...  0.0  0.0  0.0       0.0
4  0.055283  0.0   0.0         0.0  ...  0.0  0.0  0.0       0.0

[5 rows x 35788 columns]

 Testing features 

      00  000  0000  0000001200  000005102000  ...  zyxel   zz  zzz   ªl  íålittin
0  0.0  0.0   0.0         0.0           0.0  ...    0.0  0.0  0.0  0.0       0.0
1  0.0  0.0   0.0         0.0           0.0  ...    0.0  0.0  0.0  0.0       0.0
2  0.0  0.0   0.0         0.0           0.0  ...    0.0  0.0  0.0  0.0       0.0
3  0.0  0.0   0.0         0.0           0.0  ...    0.0  0.0  0.0  0.0       0.0
4  0.0  0.0   0.0         0.0           0.0  ...    0.0  0.0  0.0  0.0       0.0

[5 rows x 35788 columns]


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [86]:
clf = LogisticRegression()
clf.fit(X_train_tfidf, twenty_train.target)
pred = clf.predict(X_test_tfidf)
report = metrics.classification_report(twenty_test.target, pred,
        target_names=twenty_test.target_names)
print(report)
print("accuracy: {:0.3f}".format(metrics.accuracy_score(twenty_test.target, pred)))

                        precision    recall  f1-score   support

           alt.atheism       0.96      0.75      0.85       319
         comp.graphics       0.83      0.97      0.89       389
               sci.med       0.93      0.86      0.89       396
soc.religion.christian       0.87      0.94      0.90       398

              accuracy                           0.89      1502
             macro avg       0.90      0.88      0.88      1502
          weighted avg       0.89      0.89      0.89      1502

accuracy: 0.887
